# Context Augmented Generation (CAG) Demo
In this notebook, we demonstrate the CAG approach: **Context Augmented Generation**.

Instead of searching a database, we fit the entire relevant context into the model's Prompt Window. 
This is faster to implement but limited by the model's context size (16k in your docker-compose).

In [1]:
import ollama

# Configuration
# We use a small model for speed in this demo.
MODEL = "qwen3:0.6b"

# Let's verify the model is available
try:
    ollama.chat(model=MODEL, messages=[{'role': 'user', 'content': 'hi'}])
    print(f"Model {MODEL} is ready.")
except Exception as e:
    print(f"Model not found. Pulling {MODEL}...")
    ollama.pull(MODEL)
    print("Model pulled.")

Model qwen3:0.6b is ready.


In [2]:
# 1. Define the Context
# This simulates a specific document (e.g., a specific internal memo or a small manual)

context_text = """
UNIVERSITY OF AVEIRO (UA) FACT SHEET 2024

Department of Electronics, Telecommunications and Informatics (DETI):
The DETI is one of the largest departments at UA. It was founded in 1974.
The current director is Prof. Nuno Borges Carvalho.
The department hosts several research units including the Institute of Telecommunications (IT) and IEETA.

Cafeteria Menu for Tuesday:
- Dish A: Codfish with Cream
- Dish B: Tofu Stir Fry
- Soup: Caldo Verde

Network Access Policy:
Students must use the 'eduroam' network. The password expires every 90 days.
Guest access is provided via the 'UA-Guest' portal which requires SMS authentication.
"""

In [3]:
# 2. The CAG Query Function
# We simply paste the text into the system prompt or user message.

def ask_cag(question, context):
    prompt = f"""
    Use the following context to answer the question. Do not use outside knowledge.

    CONTEXT:
    {context}

    QUESTION:
    {question}
    """

    response = ollama.chat(model=MODEL, messages=[
        {'role': 'user', 'content': prompt}
    ])

    return response['message']['content']

In [4]:
# 3. Run Inferences

q1 = "Who is the director of DETI?"
print(f"Q: {q1}")
print(f"A: {ask_cag(q1, context_text)}\n")

q2 = "How do I connect to the wifi if I am a visitor?"
print(f"Q: {q2}")
print(f"A: {ask_cag(q2, context_text)}\n")

q3 = "Can I eat meat on Tuesday?"
print(f"Q: {q3}")
print(f"A: {ask_cag(q3, context_text)}")

Q: Who is the director of DETI?
A: Prof. Nuno Borges Carvalho.

Q: How do I connect to the wifi if I am a visitor?
A: To connect to the university's WiFi as a visitor, you need to use the "UA-Guest" portal. This portal requires SMS authentication to access the network.

Q: Can I eat meat on Tuesday?
A: 法
</think>法

Yes, you can eat meat on Tuesday. The cafeteria menu lists "Codfish with Cream" as a dish, which includes meat, so you can eat meat on Tuesday. The other dishes are vegetarian, so the answer is yes.
